# 卷积神经网络识别手写数字
### 加载数据

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
sess=tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### 权重初始化
为了创建这个模型，我们需要创建大量的权重和偏置项，为了不在建立模型的时候反复做初始化操作，我们定义两个函数用于初始化。

- 权重：这个模型中的权重在初始化时应该加入少量的噪声来打破对称性以及避免 0 梯度，因此标准差设为 0.1。
- 偏置：由于我们使用的是ReLU神经元，因此比较好的做法是用一个较小的正数来初始化偏置项，以避免神经元节点输出恒为0的问题（dead neurons）。

In [5]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

### 卷积和池化

TensorFlow 在卷积和池化上有很强的灵活性。我们怎么处理边界？步长应该设多大？在这个实例里，我们会一直使用 vanilla 版本。我们的卷积使用1步长（stride size），0边距（padding size）的模板，保证输出和输入是同一个大小。我们的池化用简单传统的2x2大小的模板做 max pooling。为了代码更简洁，我们把这部分抽象成一个函数。

- tf.nn.conv2d 是 TensorFlow 中的 2 维卷积函数，其中 x 是输入，W 是卷积的参数，Strides 代表卷积模板移动的步长，Padding 代表边界的处理方式，padding='SAME' 表明不再给输入元素的周围补充元素，让卷积的输入和输出保持同样的尺寸。具体示例参见零基础入门深度学习(4) - 卷积神经网络。
- tf.nn.max_pool 是 TensorFlow 中的最大池化函数，在这里使用 2x2 的最大池化，即将一个 2x2 的像素块降为 1x1 的像素。最大池化会保留原始像素块中灰度值最高的那一个像素，即保留最显著特征。池化层的 strides 设为横竖两个方向以 2 为步长。

In [6]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

### 第一层卷积

首先定义输入的 placeholder，x 是特征，y_ 是真实的 label。因为卷积神经网络会利用到空间结构信息，因此需要将 1D 的输入向量转为 2D 的图片结构，即从 1x784 的形式转为原始的 28x28 的结构。同时因为只有一个颜色通道，故最终尺寸为[-1,28,28,1]，前面的 -1 代表样本数量不固定，最后的 1 代表颜色通道数为 1（因为是灰度图所以这里的通道数为 1，如果是 rgb 彩色图，则为 3）。这里我们使用的 tensor 变形函数是 tf.reshape。

In [9]:
x=tf.placeholder(tf.float32,[None,784])
y_=tf.placeholder(tf.float32,[None,10])
x_image=tf.reshape(x,[-1,28,28,1])

现在我们可以开始实现第一层了。首先使用前面写好的函数进行参数初始化，包括 weights 和 bias。

- weights：卷积的权重张量形状是 [5, 5, 1, 32]，前两个维度是patch的大小，接着是输入的通道数目，最后是输出的通道数目，即卷积核尺寸是 5x5，颜色通道是 1，有 32 个不同的卷积核。
- bias：卷积在每个 5x5 的 patch 中算出 32 个特征，而对于每一个输出通道都有一个对应的偏置量。

第一层卷积由一个卷积接一个 max pooling 完成：

- 首先使用 conv2d 函数进行卷及操作，并加上偏置，接着再使用 ReLU 激活函数进行非线性处理。
- 然后使用最大池化函数 max_pool_2x2 对卷积的输出结果进行池化操作。


In [10]:
W_conv1=weight_variable([5,5,1,32])
b_conv1=bias_variable([32])

h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool_2x2(h_conv1)

### 第二层卷积

这个卷积层基本和第一个卷积层一样，唯一不同的是，卷积核的数量变成了 64，也就是说每个 5x5 的patch 会得到 64 个特征。

In [11]:
W_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])

h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)

### 密集连接层

因为前面经历了两次步长为 2x2 的最大池化，所以边长只有 1/4 了，即图片尺寸由 28x28 变为 7x7。并且由于第二个卷积层的卷积核数量为 64，所以输出的 tensor 尺寸是 7x7x64。

我们加入一个有 1024 个神经元的全连接层，用于处理整个图片。我们把池化层输出的张量 reshape 成一些向量，乘上权重矩阵，加上偏置，然后对其使用 ReLU。

In [12]:
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])

h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

### Dropout

为了减少过拟合，我们在输出层之前加入 Dropout。

我们用一个 placeholder 来代表一个神经元的输出在 dropout 中保持不变的概率。这样我们可以在训练过程中启用 dropout，在测试过程中关闭 dropout。TensorFlow 的 tf.nn.dropout 操作除了可以屏蔽神经元的输出外，还会自动处理神经元输出值的 scale。所以用 dropout 的时候可以不用考虑 scale。

In [13]:
keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

### 输出层

最后，我们添加一个softmax层，就像前面的单层softmax regression一样。

In [14]:
W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

这里的损失函数依然使用交叉信息熵，优化器使用 Adam，并把学习速率设为较小的 1e-4。

再继续定义评测准确率的操作，这里和之前一样。

In [17]:
cross_entropy=-tf.reduce_sum(y_*tf.log(y_conv))
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

下面开始训练过程，首先依然是初始化所有参数。

keep_prob 在训练时设置为 0.5。这里采用 5000 个 batch，每个 batch 包含 50 条的样本，参与训练的样本量共 25 万。其中每 500 次训练，会对准确率进行一次测评（测评时  keep_prob 设为 1），用以检测模型的性能。

In [19]:
tf.global_variables_initializer().run()
for i in range(20000):
    batch=mnist.train.next_batch(50)
    if i%1000==0:
        train_accuracy=accuracy.eval({x:batch[0],y_:batch[1],keep_prob:1.0})
        print 'step {},training accuracy {}'.format(i,train_accuracy)
    train_step.run({x:batch[0],y_:batch[1],keep_prob:0.5})

step 0,training accuracy 0.20000000298
step 1000,training accuracy 0.939999997616
step 2000,training accuracy 0.959999978542
step 3000,training accuracy 0.980000019073
step 4000,training accuracy 1.0
step 5000,training accuracy 1.0
step 6000,training accuracy 1.0
step 7000,training accuracy 0.980000019073
step 8000,training accuracy 1.0
step 9000,training accuracy 1.0
step 10000,training accuracy 1.0
step 11000,training accuracy 1.0
step 12000,training accuracy 1.0
step 13000,training accuracy 1.0
step 14000,training accuracy 1.0
step 15000,training accuracy 1.0
step 16000,training accuracy 1.0
step 17000,training accuracy 1.0
step 18000,training accuracy 1.0
step 19000,training accuracy 1.0


全部训练完成后，在测试集上进行全面的测试，得到分类的准确率。

In [20]:
test_accuracy=accuracy.eval({x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0})
print 'test accuracy',test_accuracy

test accuracy 0.9918
